<a href="https://colab.research.google.com/github/thanhlong1997/100-nlp-papers/blob/master/mlp_classifier_v27_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/vinai/phobert-base/vocab.txt

--2020-10-13 08:11:37--  https://s3.amazonaws.com/models.huggingface.co/bert/vinai/phobert-base/vocab.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.141.206
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.141.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 895321 (874K) [text/plain]
Saving to: ‘vocab.txt’

vocab.txt           100%[===================>] 874.34K   966KB/s    in 0.9s    

2020-10-13 08:11:38 (966 KB/s) - ‘vocab.txt’ saved [895321/895321]



In [ ]:
def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char

    pairs = set(pairs)
    return pairs
print(get_pairs('Hôm_nay'))

{('n', 'a'), ('m', '_'), ('ô', 'm'), ('a', 'y'), ('_', 'n'), ('H', 'ô')}


In [ ]:
with open('bpe.codes', encoding="utf-8") as merges_handle:
  merges = merges_handle.read().split("\n")[:-1]

merges = [tuple(merge.split()[:-1]) for merge in merges]
bpe_ranks = dict(zip(merges, range(len(merges))))
cache = {}

def bpe(token):
  if token in cache:
    return cache[token]
  word = tuple(token)
  word = tuple(list(word[:-1]) + [word[-1] + "</w>"])
  pairs = get_pairs(word)

  if not pairs:
    return token

  while True:
    bigram = min(pairs, key=lambda pair: bpe_ranks.get(pair, float("inf")))
    if bigram not in bpe_ranks:
      break
    first, second = bigram
    new_word = []
    i = 0
    while i < len(word):
      try:
        j = word.index(first, i)
      except ValueError:
        new_word.extend(word[i:])
        break
      else:
        new_word.extend(word[i:j])
        i = j

      if word[i] == first and i < len(word) - 1 and word[i + 1] == second:
        new_word.append(first + second)
        i += 2
      else:
        new_word.append(word[i])
        i += 1
    new_word = tuple(new_word)
    word = new_word
    if len(word) == 1:
      break
    else:
      pairs = get_pairs(word)
  word = "@@ ".join(word)
  word = word[:-4]
  cache[token] = word
  return word

print(bpe('Viblo'))

Vi@@ b@@ lo


In [ ]:
import re
def _tokenize(text):
    """Tokenize a string."""
    split_tokens = []

    words = re.findall(r"\S+\n?", text)

    for token in words:
        split_tokens.extend([t for t in bpe(token).split(" ")])
    return split_tokens

print(_tokenize('Hôm_nay trời nóng quá nên tôi ở nhà viết Viblo!'))

['Hôm_nay', 'trời', 'nóng', 'quá', 'nên', 'tôi', 'ở', 'nhà', 'viết', 'Vi@@', 'blo@@', '!']


In [ ]:
encoder = {}
bos_token="<s>"
eos_token="</s>"
sep_token="</s>"
cls_token="<s>"
unk_token="<unk>"
pad_token="<pad>"
mask_token="<mask>"


encoder[bos_token] = 0
encoder[pad_token] = 1
encoder[eos_token] = 2
encoder[unk_token] = 3

def add_from_file(f):
  if isinstance(f, str):
      try:
          with open(f, "r", encoding="utf-8") as fd:
              add_from_file(fd)
      except FileNotFoundError as fnfe:
          raise fnfe
      except UnicodeError:
          raise Exception("Incorrect encoding detected in {}, please " "rebuild the dataset".format(f))
      return

  lines = f.readlines()
  for lineTmp in lines:
      line = lineTmp.strip()
      idx = line.rfind(" ")
      if idx == -1:
          raise ValueError("Incorrect dictionary format, expected '<token> <cnt>'")
      word = line[:idx]
      encoder[word] = len(encoder)

add_from_file('vocab.txt')

In [ ]:

def _convert_token_to_id(token):
    """ Converts a token (str) in an id using the vocab. """
    return encoder.get(token, encoder.get(unk_token))
print(_convert_token_to_id('Hôm_nay'))

3791


In [ ]:
decoder = {v: k for k, v in encoder.items()}
def _convert_id_to_token(index):
    return decoder.get(index, unk_token)

print(_convert_id_to_token(7))

của


In [ ]:
def convert_tokens_to_string(tokens):
    out_string = " ".join(tokens)
    out_string = out_string.replace("@@ ", "").strip()
    out_string = out_string.replace('@@','').strip()
    return out_string
  
print(convert_tokens_to_string(['Hôm_nay', 'trời', 'nóng', 'quá' ,'nên', 'tôi', 'ở', 'nhà', 'viết', 'Vi@@', 'blo@@']))

Hôm_nay trời nóng quá nên tôi ở nhà viết Viblo


# Requirement



In [ ]:
!pip install pandas
  

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 2.8MB/s 
     |████████████████████████████████| 2.9MB 15.2MB/s 
     |████████████████████████████████| 890kB 39.1MB/s 
     |████████████████████████████████| 1.1MB 37.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=71f61c09a7f6d2c02577b42978d50f6a641bc64140901f28cf91c616f6a7f83b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
! pip install torch torchvision

In [ ]:
! pip3 install vncorenlp

     |████████████████████████████████| 2.7MB 2.8MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp36-none-any.whl size=2645934 sha256=fe9207968868e12fd70ec0fe7746589ea7ef6292667c8e334b833140f1973fa7
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp


In [ ]:
# ! rm -rf 

In [ ]:
! mkdir -p vncorenlp/models/wordsegmenter
! wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
! wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
! wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
! mv VnCoreNLP-1.1.1.jar vncorenlp/ 
! mv vi-vocab vncorenlp/models/wordsegmenter/
! mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

--2020-10-27 01:11:22--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  34.3MB/s    in 0.8s    

2020-10-27 01:11:24 (34.3 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2020-10-27 01:11:24--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200

In [ ]:
from vncorenlp import VnCoreNLP
import re

rdrsegmenter = VnCoreNLP("/content/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 
def document_2_list_token(documents):
  vocab = {}
  for document in documents:
    tokenizer = rdrsegmenter.tokenize(document)
    for sentence in tokenizer:
      for token in sentence:
        token = token.lower()
        if not re.search('[a-zA-Z]',token):
          continue
        if token != '':
          if token not in vocab:
            vocab[token] = 1
          else:
            vocab[token] += 1
  return vocab


In [ ]:
print(rdrsegmenter.tokenize('ban biet thu duong hoa phuong 1 son tra'))


[['ban', 'biet', 'thu', 'duong', 'hoa', 'phuong', '1_son', 'tra']]


# Entity Feature


In [ ]:
import  torch
from transformers import AutoModel, AutoTokenizer
from vncorenlp import VnCoreNLP
import numpy as np

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
def get_token_line(text):
  tokens_line = rdrsegmenter.tokenize(text)
  line = " "
  sentence = ''
  for line in tokens_line:
    line = ' '.join(token for token in tokens_line[0])
    sentence += line + ' '
    line = ' '
  return sentence

def get_token_list(text):
  tokens_line = rdrsegmenter.tokenize(text)
  sentence = []
  for line in tokens_line:
   sentence.extend(line)
  return sentence

def get_input_ids(text):
  sentence = get_token_line(text)
  input_id = torch.tensor([tokenizer.encode(sentence)])
  return input_id

def get_features_senten(text):
  input_id = get_input_ids(text)
  with torch.no_grad():
      features = phobert(input_id)  
  cls_token = features[0][0][0]
  pooling = np.zeros(768)
  for token in features[0][0]:
    pooling = np.add(pooling,token)
  return cls_token,pooling/(len(input_id))

In [ ]:
from scipy import spatial
def similar_2_sen(text1 , text2, is_pooling = True):
  text1 = text1.lower()
  text2 = text2.lower()
  cls_token1, pooling1 = get_features_senten(text1)
  cls_token2, pooling2 = get_features_senten(text2)
  if is_pooling:
    vec1 = pooling1
    vec2 = pooling2
  else:
    vec1 = cls_token1
    vec2 = cls_token2
  dataSetI = vec1.tolist()
  dataSetII = vec2.tolist()
  result = 1 - spatial.distance.cosine(dataSetI, dataSetII)
  return result

In [ ]:
class_construction_land = ['là đất thổ cư','đất ở','trong đó thổ cư','đất ở lâu dài','thổ cư','diện tích xây dựng','diện tích sử dụng','nhà rộng','nhà xây','ont','diện tích đất ở','dt đất ở','dt sử dụng','dt nhà','diện tích nhà']
other_type_land = ['diện tích vườn','diện tích ao','diện tích nông nghiệp','diện tích mái','diện tích sản xuất','đất trồng cây lâu năm']
# other_type_land = []
total_land = ['lô đất','tổng dt','tổng diện tích']
# total_land =[]

def computing_max_ability(text,class_area):
  text = text.lower()
  max = 0
  context_ability = ''
  class_patent = []
  if class_area == 1:
    class_patent = class_construction_land
  if class_area == 2:
    class_patent = other_type_land
  if class_area == 3:
    class_patent = total_land
  for item in class_patent:
    item = get_token_list(item)
    item = ' '.join(x for x in item)
    similar = similar_2_sen(text,item)
    # print(text ,'------', item,'-----',similar)
    if similar > max :
      max = similar
      context_ability = item
  # print(context_ability)
  return max 

In [ ]:
import re
def  clean_text(text):
  text = re.sub('[:\'"|`~]','',text)
  return text


# print(clean_text('diện tích :'))
class classifier_area:
  def __init__(self, ner_result):
    self.input = ner_result
  
  def get_position_of_area(self):
    result = []
    for item in self.input['labels']:
      if item[2] == 'size':
        result.append(item)  
    return result
      
  def get_context_of_entity(self):
    context_menu = []
    position_area_entity = self.get_position_of_area()
    for item in position_area_entity:
      context_before = []
      context_menu.append([item[0] , item[1]])
#----------------------------------------------------------------      
      if position_area_entity.index(item) > 0:
        index_before = position_area_entity.index(item)
        text_before = self.input['text'][position_area_entity[index_before - 1][1]:item[0]]
        text_before = clean_text(text_before)
      else:
        text_before =  self.input['text'][:item[0]]
      text_before = get_token_list(text_before)
      text_before = list(text_before)
      text_before.reverse()
      # print((text_before))
      end_senten_pattent = ['.',',','?','!','-','+',')','(','...']
      if len(text_before) > 0:
        if text_before[0] not in end_senten_pattent:
          for token in text_before:
            if token not in end_senten_pattent and len(context_before) < 3:
              context_before.append(token)
            else:
              break
      context_before.reverse()
      if len(context_before) > 0:
        context_menu[-1].append(' '.join(x for x in context_before))
# -----------------------------------------------------------------------
      context_after = []
      if position_area_entity.index(item) < len(position_area_entity)-1:
        index_after = position_area_entity.index(item)
        text_after = self.input['text'][item[1]:position_area_entity[index_after+1][0]]
        text_after = clean_text(text_after)
      else:
        text_after = self.input['text'][item[1]:]
      text_after = list(get_token_list(text_after))
      # print(text_after)
      if len(text_after) > 0:
        if text_after[0] not in end_senten_pattent:
          for token in text_after:
            if token not in end_senten_pattent and len(context_after) < 3:
              context_after.append(token)
            else:
              break
      if len(context_after) > 0:
        context_menu[-1].append(' '.join(x for x in context_after))
    return context_menu 

  def computing_similar(self):
    labels = []
    contexts = self.get_context_of_entity()
    for context in contexts:
      max = 0
      classs = 0
      context_text = context[2:]
      # print(context_text)
      for text in context_text:
        for i in range(1,4):
          computed = computing_max_ability(text,i)
          # print(computed)
          if max < computed:
            max = computed
            classs = i

      labels.append(list([classs,max]))
    return labels  

In [ ]:
import re

def get_number_floor(data):
  map = {'một':'1', 'hai':'2', 'ba':'3', 'bốn':'4', 'năm':'5', 'sáu':'6', 'bảy':'7', 'tám':'8', 'chín':'9', 'không':'0'}
  max_floor = 0 
  text = ''
  for item in data['labels']:
    if 'floor' in item[2].split('_') :
      # print(data['text'][item[0]:item[1]])
      text = data['text'][item[0]:item[1]]
      transform = ''
      for item in text.split(' '):
        if item in map : 
          transform += ' ' + map[item]
        else:
          transform +=' ' + item
      text = transform
      if re.search('(\d+\D\d+)',text):
        floor_patent = re.search('(\d+\D\d+)',text).group()
        if re.search('\.|,',floor_patent):
          floor = float(re.sub('\D','.',floor_patent))
        else:
          floor = [float(x) for x in re.split('\D+',floor_patent)]
        # floor = [int(x) for x in floor]
          floor =  max(floor)
        if max_floor < floor:
          max_floor = floor
      else:
        floor = text.split(' ')
        for token in floor:
          token = re.sub('\D','',token)
          if token.isdigit():
            if float(token) > max_floor:
              max_floor = float(token)
        # print('floor',max_floor)
        # if max_floor < floor:
        #   max_floor = floor

  if text == '':
    return 0
  return float(max_floor)

In [ ]:
import re
def area_process(input):
  classifier = classifier_area(input)
  ability = classifier.computing_similar()
  position = classifier.get_position_of_area()
  # print(ability)
  # print(position)
  # 1/0
  number_floor = get_number_floor(input)
  if number_floor == 0:
    number_floor =1
  area = {'use_area':0,'total_land_area':0,'contruction_area':0}
  area_content =  []
  for item in position:
    area_content.append(input['text'][item[0]:item[1]])
  # print(area_content)

  numberic_value =[]
  for item in area_content:
    item = item.lower()
    if re.search('\d*[\.*,*]*\d+\s*(m|ha)',item):
      value = re.search('\d*[\.*,*]*\d+\s*(m|ha)',item).group()
      value = re.sub('m2','',value)
      value = re.sub('ha','',value)
      value = re.sub('m','',value)
      value = re.sub(',','.',value)
      # print(value)
      # 1/0
      try:
        value = float(value)
      except:
        value = re.sub('\D','',value)
      numberic_value.append(float(value))

  max_consistent_construction = 0
  construction_value = 0
  # print((numberic_value))
  for index in range(len(numberic_value)):
    # print(index)
    classified = list(ability[index])
    # print(classified)
    # 1/0
    if classified[0] ==  1:
      if max_consistent_construction < classified[1] or (max_consistent_construction == classified[1] and numberic_value[index] < construction_value):
        max_consistent_construction = classified[1]
        construction_value = numberic_value[index]
  # print(max_consistent_construction,construction_value)
  max_total_area = 0
  for index in range(len(numberic_value)):
    classified = list(ability[index])
    if numberic_value[index] > construction_value:
    # if classified[1] < 0.53:
      if max_total_area < numberic_value[index] and numberic_value[index] != number_floor*construction_value:
        max_total_area = numberic_value[index]
    # if classified[0] == 1 and max_total_area < numberic_value[index] and numberic_value[index] != number_floor*construction_value:
      # max_total_area = numberic_value[index]
  # print(max_total_area)
  if max_total_area == 0:
    max_total_area = construction_value
  if construction_value == 0:
    construction_value = max_total_area
  # print('num floor:' , number_floor)
  area['use_area'] = construction_value*number_floor
  area['total_land_area'] = max_total_area
  area['contruction_area'] = construction_value
  return area

In [ ]:
def combine_house_entity(input):
  area = area_process(input)
  number_floor = get_number_floor(input)
  slot = {'house_sizes':area,'house_description':{'floor':0,'mat_tien':'','size_mat_ngo':'','number_livingroom':'','number_kitchen':'','number_dressroom':'','number_bedroom':''},
          'land_description':{},'price':0,'house_locate':{'near_places':'','location':''},'contact':{'contact_mobile':'','email':''}}
  for item in input['labels']:
    # print(item[2])
    if item[2] in slot['house_description']:
      if  slot['house_description'][item[2]] == '' or slot['house_description'][item[2]] == 0:
        slot['house_description'][item[2]] = input['text'][item[0]:item[1]]
      # print(slot['house_description'][item[2]])
      # 1/0
  
    if item[2] in slot['house_locate']:
      slot['house_locate'][item[2]] += input['text'][item[0]:item[1]] + ' '
  
    if item[2] in slot['contact']:
      slot['contact'][item[2]] = input['text'][item[0]:item[1]]

    if item[2] in slot:
      slot[item[2]] = input['text'][item[0]:item[1]]
  
  slot['house_description']['floor'] = number_floor
  # print(slot)
  return slot

In [ ]:
print(combine_house_entity(data[0]))

{'house_sizes': {'use_area': 0, 'total_land_area': 0, 'contruction_area': 0}, 'house_description': {'floor': 0, 'mat_tien': '4m', 'size_mat_ngo': '', 'number_livingroom': '', 'number_kitchen': '', 'number_dressroom': '', 'number_bedroom': ''}, 'land_description': {}, 'price': 0, 'house_locate': {'near_places': 'đường Quốc Lộ 21B xóm 8 thông Đinh Xuyên Xã Hòa Nam ', 'location': ''}, 'contact': {'contact_mobile': '0914 415 ***', 'email': ''}}


# Content Feature

In [ ]:
import json

with open('a.json','r',encoding='utf-8') as file:
  data = json.load(file)
print(len(data))

4752


In [ ]:
import pandas as pd
import math
from itertools import islice

dfs = pd.read_excel('data_classifier.xlsx', sheet_name='Sheet1')

check_nan_land = dfs['land'].isnull()
colum_land = list(dfs['land'])
check_nan_house = dfs['house'].isnull()
colum_house = list(dfs['house'])

index_land = []
index_house = []

for index1 in range(len(colum_land)):
  if not check_nan_land[index1]:
    index_land.append(int(colum_land[index1]))

for index1 in range(len(colum_house)):
  if not check_nan_house[index1]:
    index_house.append(int(colum_house[index1]))

data_land = [data[index]['text'] for index in index_land]
data_house = [data[index]['text'] for index in index_house]


count_feature_land = document_2_list_token(data_land)
count_feature_land = {k: v for k, v in sorted(count_feature_land.items(), key=lambda item: item[1], reverse=True)}

# data_house = data_house[:index2]
count_feature_house = document_2_list_token(data_house)
count_feature_house = {k: v for k, v in sorted(count_feature_house.items(), key=lambda item: item[1], reverse=True)}

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))
print(take(50,count_feature_land.items()))
print(take(50,count_feature_house.items()))

[('có', 200), ('đất', 190), ('nhà', 139), ('bán', 109), ('cách', 82), ('đường', 81), ('giá', 74), ('chính', 67), ('chủ', 66), ('sổ_đỏ', 58), ('cấp', 57), ('khu', 57), ('gần', 55), ('và', 54), ('rất', 53), ('liên_hệ', 52), ('diện_tích', 51), ('cần', 47), ('lô', 42), ('hà_nội', 41), ('cho', 40), ('tỷ', 39), ('ô_tô', 38), ('đẹp', 38), ('trong', 36), ('chỉ', 36), ('nhà_đất', 35), ('nhu_cầu', 35), ('lại', 35), ('là', 33), ('vị_trí', 32), ('vuông_vắn', 32), ('được', 32), ('pháp_lý', 30), ('lh', 30), ('đầy_đủ', 27), ('rộng', 27), ('thuận_tiện', 27), ('m', 26), ('không', 26), ('làm', 26), ('tầng', 26), ('mảnh', 25), ('chợ', 25), ('thông_tin', 25), ('tại', 25), ('mặt_tiền', 24), ('vào', 24), ('an_ninh', 24), ('m2', 24)]
[('nhà', 363), ('tầng', 299), ('phòng', 284), ('có', 143), ('cách', 136), ('chính', 122), ('gần', 119), ('giá', 112), ('ngủ', 112), ('tỷ', 110), ('sổ_đỏ', 108), ('bán', 106), ('chủ', 105), ('và', 104), ('đường', 99), ('bếp', 95), ('sân', 90), ('khu', 87), ('khách', 85), ('cửa', 

In [ ]:
vocabulary_land = ['đất','nhà','lô','nhà_đất','vuông_vắn','vườn','mảnh','xưởng','kho']
vocabulary_house = ['tầng','phòng','ngủ','bếp','khách','sân','thiết_kế','phơi','thờ','nội_thất','wc','hướng',',mặt_tiền','vệ_sinh','cửa','ban_công','tiện_nghi']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import numpy as np
from scipy import sparse

def tokenize(text):
  tokens = []
  document = rdrsegmenter.tokenize(text)
  for senten in document:
    for token in senten:
      tokens.append(token.lower())
  return tokens

def transform_document_to_vec(input):
  text = input['text']
  vocab = vocabulary_land
  for item in vocabulary_house:
    if item not in vocab:
      vocab.append(item)
  
  text = text.lower()
  vectorizer = CountVectorizer(tokenizer=tokenize,vocabulary=vocab,binary=True)
  # print(vectorizer.build_tokenizer())
  # 1/0
  X = vectorizer.fit_transform([text]).toarray()
  return X

def transform_entity_to_vec(input):
  slot = combine_house_entity(input)
  vec = np.zeros(7)
  for item in slot['house_description']:
    if slot['house_description'][item] not in ['',0]:
      vec[list(slot['house_description'].keys()).index(item)] = 1
  return vec , slot

# print(transform_entity_to_vec(data[1001]))
# 1/0
def convert_labels(y, C = 2):
    Y = sparse.coo_matrix((np.ones_like(y),
        (y, np.arange(len(y)))), shape = (C, len(y))).toarray()
    return Y.T

# a = transform_document_to_vec('Bán nhà 40m2x 3 tầng mới đường oto đỗ cửa giá 1.38 tỷ có TL ... Hướng : Đông Nam ... Diện tích : 40m2 nở hậu ... Vị trí : Nhà có vị trí cực thuận tiện gần trường học các cấp , công an phường , quân đội , nhà văn hóa . Nhà gần bãi gửi xe ngày đêm , đường trước nhà rộng 3 cực thoáng trong dãy phân lô cao tầng ... + Nhà gần trục đường 6 tuyến đi Hà Đông , Ngã Tư Sở , Mỹ Đình hay tuyến Chương Mỹ , Hòa Bình ... + Nhà gần nhiều dự án trọng đểm của quận Hà Đông là khu đang phát triển rất mạnh .. ... + Khu dân cư đông , an ninh tốt , hàng xóm thân thiện .. ... + Nhà gần điểm xe bus cách đường quốc lộ 6 chỉ 500m ... + Với tài chính vừa phải bạn sở hữu căn nhà lý tưởng ... Thiết kế : Theo kiến trúc tân cổ điển , rộng , thoáng , tầng 2 phòng ngủ ... + Tâng 1 : Phòng khách , để xe , bếp ăn riêng , có sân sau ... + Tầng 2 : 2 phòng ngủ rộng , 1 wc ... + Tầng 3 : 1 phòng ngủ , 1 phòng thờ , sân phơi rộng ... + Nhà có sân trước 3m , sân sau 1,5m các phòng đều có cửa sổ rộng , không khí lưu thông ... Nội thất : Hoàn thiện đầy đủ nội thất cơ bản , nhận nhà ngay ... Pháp lý : Giấy phép xây dựng riêng ... Sổ đỏ chính chủ ... Giá : 1.38 tỷ làm việc trực tiếp , bao sang tên ... Liên hệ : Tuấn Anh 0974322298 đi xem miễn phí ... Hỗ trợ thủ tục vay vốn ngân hàng 70% giá trị với lãi xuất thấp , thủ tục nhanh ... Thông tin pháp lý : Sổ đỏ chính chủ.')
# print(a.shape)
Feature_1 = []
Feature_2 = []
label = []
for index in index_house:
  print(index)
  Feature_1.extend(transform_document_to_vec(data[index]))
  Feature_2.append(transform_entity_to_vec(data[index])[0])
  label.append(0)
for index in index_land:
  print(index)
  Feature_1.extend(transform_document_to_vec(data[index]))
  Feature_2.append(transform_entity_to_vec(data[index])[0])
  label.append(1)
Feature_1 = np.asarray(Feature_1)
Feature_2 = np.asarray(Feature_2)
Label     = np.asarray(label)
print(Feature_2)


203
204
205
206
207
208
210
287
288
289
291
290
292
293
294
295
296
297
299
298
301
302
304
317
328
323
330
336
332
337
338
68
70
72
73
76
84
85
86
87
89
90
91
343
348
350
351
352
356
258
361
360
362
366
370
373
377
383
384
387
392
394
395
399
403
406
407
408
411
413
414
415
417
418
419
420
421
422
424
425
427
428
430
431
433
436
437
438
441
443
447
452
453
454
463
468
482
486
488
202
209
307
329
1
6
5
7
8
12
13
19
20
21
22
23
32
34
39
40
42
45
46
47
48
52
53
54
56
57
58
59
60
61
65
63
64
67
66
70
77
78
80
355
363
364
382
388
390
391
412
4719
4709
4640
4575
4447
4431
4386
4168
3984
3896
3868
3860
3830
3669
3657
3639
3555
3412
3374
3367
3318
3254
3242
3045
2972
2957
2871
2870
2865
2834
2804
2709
2605
2518
[[1. 1. 1. ... 1. 1. 1.]
 [1. 0. 1. ... 1. 1. 1.]
 [1. 0. 1. ... 1. 1. 1.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 1.]
 [0. 0. 1. ... 0. 0. 0.]]


In [ ]:
# save feature for future use
import numpy as np
np.savez('123.npz', a=Feature_1, b=Feature_2, c=Label)

In [ ]:
print(transform_entity_to_vec(data[0]))

(array([0., 1., 0., 0., 0., 0., 0.]), {'house_sizes': {'use_area': 0, 'total_land_area': 0, 'contruction_area': 0}, 'house_description': {'floor': 0, 'mat_tien': '4m', 'size_mat_ngo': '', 'number_livingroom': '', 'number_kitchen': '', 'number_dressroom': '', 'number_bedroom': ''}, 'land_description': {}, 'price': 0, 'house_locate': {'near_places': 'đường Quốc Lộ 21B xóm 8 thông Đinh Xuyên Xã Hòa Nam ', 'location': ''}, 'contact': {'contact_mobile': '0914 415 ***', 'email': ''}})


In [ ]:
# Load Feature saved
Feature = np.load('123.npz',allow_pickle=True)
a = Feature['a']
b = Feature['b']
c = Feature['c']
print(b)

index_array = range(a.shape[0])

index_train, index_test = train_test_split(
     index_array, test_size=0.33, random_state=42)
# print(a[[1,2,3,4]])
# print(index_train)
# 1/0
Feature1_train = a[index_train]
Feature1_test = a[index_test]

Feature2_train = b[index_train]
Feature2_test = b[index_test]
y_train = c[index_train]
y_test = c[index_test]

y_train =  convert_labels(y_train)
y_test  =  convert_labels(y_test)
print(Feature2_train.shape)
print(Feature1_train.shape)

In [ ]:
print(len(y_train),len(y_test))

123 61


In [ ]:
print(Feature1_train.shape , Feature2_test.shape)

(123, 26) (61, 7)


In [ ]:
print(Feature2_train)

[[1. 1. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 1. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 0. 1. 1.]
 [1. 0. 0. 1. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 0.]
 [1. 0. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 0. 1.]
 [1. 0. 0. 1. 0. 1. 1.]
 [1. 1. 1. 0. 0. 0. 0.]
 [1. 0. 0. 1. 1. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 1. 1.]
 [1. 0. 0. 1. 1. 1. 1.]
 [1. 0. 1. 0. 0. 0. 1.]
 [1. 0. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 1. 1. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 1. 1. 0. 1.]
 [1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 0. 1. 1. 1. 1.]
 [1. 0. 0. 1. 1.

In [ ]:
! git clone https://github.com/thanhlong1997/bert_quora

Cloning into 'bert_quora'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 630 (delta 21), reused 27 (delta 10), pack-reused 592
Receiving objects: 100% (630/630), 219.91 KiB | 7.33 MiB/s, done.
Resolving deltas: 100% (399/399), done.


In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# Parameter
learning_rate = 0.001
epoch = 150
batch_size = 128
display_step =5

#Network Parameters
n_hidden_1 = 256
n_hidden_2 = 64
dimention_1 = 26 # dimention of vocabulary
dimention_2 = 7 # dimention of entities
connect_dimention = 16 # dimention connect
num_class = 2
keep_prob = 0.7
batch_size = 32

# tf Graph input
X_1 = tf.placeholder('float',[None,dimention_1],name='X_1')
X_2 = tf.placeholder('float',[None,dimention_2],name='X_2')
Y   = tf.placeholder('int64',[None,num_class],name='Y')

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
! tf_upgrade_v2 \
  --infile bert_quora/modeling.py \
  --outfile modeling_v2.py

In [ ]:
# Store layers weight and bias

weights = {
    'concat_1': tf.Variable(tf.random_normal([dimention_1,connect_dimention]),name='concat_1'),
    'concat_2': tf.Variable(tf.random_normal([dimention_2,connect_dimention]),name='concat_2'),
    'h_1'     : tf.Variable(tf.random_normal([connect_dimention,n_hidden_1]),name='h_1'),
    'h_2'     : tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2]),name='h_2'),
    'out'     : tf.Variable(tf.random_normal([n_hidden_2,num_class]),name='out')
}

biases = {
    'b_1' : tf.Variable(tf.random_normal([n_hidden_1]),name='b_1'),
    'b_2' : tf.Variable(tf.random_normal([n_hidden_2]),name='b_2'),
    'out' : tf.Variable(tf.random_normal([num_class]),name='out')
}

In [ ]:
# Create model
def neural_net(x1,x2):
  # Concat Feature
  concat  = tf.add(tf.matmul(x1,weights['concat_1']),tf.matmul(x2,weights['concat_2']))
  # hidden layer 1
  layer_1 = tf.add(tf.matmul(concat,weights['h_1']),biases['b_1'])
  layer_norm1 = tf.keras.layers.LayerNormalization(axis = -1)
  layer_1 = layer_norm1(layer_1)
  layer_1 = tf.nn.relu(layer_1)
  # hidden layer 2
  layer_2 = tf.add(tf.matmul(layer_1,weights['h_2']),biases['b_2'])
  layer_norm2 = tf.keras.layers.LayerNormalization(axis = -1)
  layer_2 = layer_norm2(layer_2)
  layer_2 = tf.nn.relu(layer_2)
  # drop_out
  drop_out = tf.nn.dropout(layer_2, keep_prob)
  # out layer
  out_layer = tf.add(tf.matmul(drop_out,weights['out']),biases['out'])
  # out_layer = tf.keras.layers.LayerNormalization(axis=1)(out_layer)
  # probabilities = tf.nn.softmax(logits, axis=-1)
  layer_norma = tf.keras.layers.LayerNormalization(axis = -1)
  out_layer = layer_norma(out_layer)
  return out_layer

In [ ]:
logit = neural_net(X_1, X_2)

# predict = tf.argmax(logit, axis=-1, output_type=tf.int32)
# log_probs = tf.nn.log_softmax(logits, axis=-1)

# one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

# per_example_loss = -tf.reduce_sum(Y * log_probs, axis=-1)
# loss_op = tf.reduce_mean(per_example_loss)
# with tf.variable_scope("loss"):
    # I.e., 0.1 dropout
    # output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    # logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    # logits = tf.nn.bias_add(logits, output_bias)
# probabilities = tf.nn.softmax(logit, axis=-1)
# log_probs = tf.nn.log_softmax(logit, axis=-1)

    # one_hot_labels = tf.one_hot(Y, depth=num_class, dtype=tf.float32)

# per_example_loss = -tf.reduce_sum(Y * log_probs, axis=-1)
# loss_op = tf.reduce_mean(per_example_loss)


# total_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_hat, y_true))
# loss 

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits = logit, labels = Y))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
import numpy as np
from sklearn.metrics import f1_score

saver = tf.train.Saver()
def training(op):
  if op == 'Adam':
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
  elif op == 'RMSprop':
    optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
  elif op == 'Momentum':
    optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate)
  train_op = optimizer.minimize(loss_op)

  predict = tf.argmax(logit, 1)
  correct_pred = tf.equal(tf.argmax(logit, 1),tf.argmax(Y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_pred , tf.float32))
  # TP = tf.count_nonzero(tf.matmul(predict , Y))
  # TN = tf.count_nonzero((predict - 1) * (Y - 1))
  # FP = tf.count_nonzero(predict * (Y - 1))
  # FN = tf.count_nonzero((predict - 1) * Y)
  # precision = TP / (TP + FP)
  # recall = TP / (TP + FN)
  # f1 = 2 * precision * recall / (precision + recall)

  loss_list = np.zeros(epoch)
  init = tf.global_variables_initializer()
  with tf.Session() as sess:
    sess.run(init)
    
    for step in range(epoch):
      batch_feature1,batch_feature2, batch_y = Feature1_train ,Feature2_train, y_train
      for i in range(0, (Feature1_train.shape[0] // batch_size) * batch_size, batch_size):
        print('epoch',step,'batch',i)
        batch_feature2 = Feature2_train[i:i+batch_size]
        batch_label = y_train[i:i+batch_size]
        loss, _ = sess.run([loss_op, train_op], 
                           feed_dict = {X_1 : np.zeros([batch_size,dimention_1]),
                                        X_2 : batch_feature2, 
                                        Y : batch_label})
        # train_loss += loss

      # sess.run(train_op, feed_dict={X_1: batch_feature1,
                                    # X_2:batch_feature2,
                                    # Y :batch_y})
      loss , acc , pred,log = sess.run([loss_op, accuracy,predict,logit],feed_dict={X_1: np.zeros([batch_feature1.shape[0],dimention_1]),
                                                                                    X_2: Feature2_train,
                                                                                    Y : y_train})
      # prop = sess.run([probabilities],feed_dict ={X_1: batch_x,
                                                  # X_2: np.zeros([batch_x.shape[0],dimention_2])} )
      loss_list[step] = loss
      if step % display_step == 0 or step == 1:
        print('Step '+str(step)+' batch loss = '+ '{:.4f}'.format(loss) + ' trainning acc ='+
              '{:.3f}'.format(acc))
        save_path = saver.save(sess, "/content/model/model" + str(step)+ ".ckpt")
        print("Model saved in path: %s" % save_path)
        batch_y_true = np.argmax(y_train, 1)
        print(batch_y_true.shape)
        print(f1_score(batch_y_true, pred, average='micro'))
        print(pred.shape)
    print('Testing Acc:',sess.run(accuracy,feed_dict={X_1: np.zeros([Feature1_test.shape[0],dimention_1]),
                                                      X_2: Feature2_test,
                                                      Y  : y_test}))
    test_result = sess.run(predict,feed_dict={X_1: np.zeros([Feature1_test.shape[0],dimention_1]),
                                              X_2: Feature2_test,
                                              Y  :y_test})
    test_true = np.argmax(y_test, 1)
    print(f1_score(test_true,test_result, average='micro'))
    print(test_result)
    print(test_true)
  return loss_list

adam = training('Adam') 

epoch 0 batch 0
epoch 0 batch 32
epoch 0 batch 64
Step 0 batch loss = 1.0335 trainning acc =0.545
Model saved in path: /content/model/model0.ckpt
(123,)
0.5447154471544715
(123,)
epoch 1 batch 0
epoch 1 batch 32
epoch 1 batch 64
Step 1 batch loss = 0.9695 trainning acc =0.577
Model saved in path: /content/model/model1.ckpt
(123,)
0.5772357723577236
(123,)
epoch 2 batch 0
epoch 2 batch 32
epoch 2 batch 64
epoch 3 batch 0
epoch 3 batch 32
epoch 3 batch 64
epoch 4 batch 0
epoch 4 batch 32
epoch 4 batch 64
epoch 5 batch 0
epoch 5 batch 32
epoch 5 batch 64
Step 5 batch loss = 0.8139 trainning acc =0.650
Model saved in path: /content/model/model5.ckpt
(123,)
0.6504065040650406
(123,)
epoch 6 batch 0
epoch 6 batch 32
epoch 6 batch 64
epoch 7 batch 0
epoch 7 batch 32
epoch 7 batch 64
epoch 8 batch 0
epoch 8 batch 32
epoch 8 batch 64
epoch 9 batch 0
epoch 9 batch 32
epoch 9 batch 64
epoch 10 batch 0
epoch 10 batch 32
epoch 10 batch 64
Step 10 batch loss = 0.5602 trainning acc =0.780
Model saved

In [ ]:
from scipy.special import softmax

tf.reset_default_graph()
n_hidden_1 = 256
n_hidden_2 = 64
dimention_1 = 26 # dimention of vocabulary
dimention_2 = 7 # dimention of entities
connect_dimention = 16 # dimention connect
num_class = 2
predict_keep_prop = 1

  # tf Graph input
X_11 = tf.placeholder('float',[None,dimention_1],name='X_11')
X_21 = tf.placeholder('float',[None,dimention_2],name='X_21')
Y1   = tf.placeholder('float',[None,num_class],name='Y1')

weights = {
    'concat_1': tf.Variable(tf.random_normal([dimention_1,connect_dimention]),name='concat_1'),
    'concat_2': tf.Variable(tf.random_normal([dimention_2,connect_dimention]),name='concat_2'),
    'h_1'     : tf.Variable(tf.random_normal([connect_dimention,n_hidden_1]),name='h_1'),
    'h_2'     : tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2]),name='h_2'),
    'out'     : tf.Variable(tf.random_normal([n_hidden_2,num_class]),name='out')
}

biases = {
    'b_1' : tf.Variable(tf.random_normal([n_hidden_1]),name='b_1'),
    'b_2' : tf.Variable(tf.random_normal([n_hidden_2]),name='b_2'),
    'out' : tf.Variable(tf.random_normal([num_class]),name='out')
}
def neural_net(x1,x2):
  # Concat Feature
  concat  = tf.add(tf.matmul(x1,weights['concat_1']),tf.matmul(x2,weights['concat_2']))
  # hidden layer 1
  layer_1 = tf.add(tf.matmul(concat,weights['h_1']),biases['b_1'])
  layer_norm1 = tf.keras.layers.LayerNormalization(axis = -1)
  layer_1 = layer_norm1(layer_1)
  layer_1 = tf.nn.relu(layer_1)
  # hidden layer 2
  layer_2 = tf.add(tf.matmul(layer_1,weights['h_2']),biases['b_2'])
  layer_norm2 = tf.keras.layers.LayerNormalization(axis = -1)
  layer_2 = layer_norm2(layer_2)
  layer_2 = tf.nn.relu(layer_2)
  # drop_out
  drop_out = tf.nn.dropout(layer_2, keep_prob)
  # out layer
  out_layer = tf.add(tf.matmul(drop_out,weights['out']),biases['out'])
  # out_layer = tf.keras.layers.LayerNormalization(axis=1)(out_layer)
  # probabilities = tf.nn.softmax(logits, axis=-1)
  layer_norma = tf.keras.layers.LayerNormalization(axis = -1)
  out_layer = layer_norma(out_layer)
  # out_layer = tf.keras.layers.LayerNormalization(axis=1)(out_layer)
  return out_layer

def predict_raw_text(input):
  # graph_meta = tf.train.import_meta_graph('/content/model/model95.ckpt.meta')
  logits = neural_net(X_11, X_21)
  probabilities = tf.nn.softmax(logits, axis=-1)
  predict = tf.argmax(logits, 1)
  vector1 = transform_document_to_vec(input)
  vector2,slot = transform_entity_to_vec(input)
  vector2 = vector2.reshape([1,7])
  with tf.Session() as sess:
    # graph_meta.restore(sess , tf.train.latest_checkpoint('/content/model'))

    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.restore(sess, '/content/model/model145.ckpt')
    pred , prop ,log = sess.run([predict,probabilities,logits],feed_dict={X_11: vector1,
                                       X_21: vector2})
  
  return pred[0], prop

# print(predict_raw_text('Bán gấp nhà Văn Điển 5 tầng ô tô đỗ gần chỉ 2,7 tỷ ... + Nhà mới đẹp Văn Điển ở ngay , ô tô đỗ gần , công năng đầy đủ cho 1 hộ gia đình ở : 1 khách , bếp , 2 ngủ , 3wc , phòng thờ , sân phơi . Có thể lên thêm 2 tầng thoải mái ... + Nhà Văn Điển gần rất nhiều tiện ích : chợ , trung tâm thương mại , bệnh viện , trung tâm thể dục thể thao , huyện ủy thanh trì ... + Khu vực Thanh Trì sắp lên Quận giá trị bất động sản ngày một tang , khách mua năm sbawts tình hình mua đợt này dịch giá có bớt cho ai thiện chí ... + Khu vực Thanh trì lên trung tâm thành phố rất gần chỉ 15 phút , đi các quận rất thuận tiện , về quê qua bến xe nước ngầm chỉ 2 phút ... + Nhà đẹp sổ vuông nở hậu ... + Liên hệ Mrs Thúy : 0965 - 2535 - 83'))
# 1/0

In [ ]:
print(predict_raw_text(data[1001]))

INFO:tensorflow:Restoring parameters from /content/model/model145.ckpt
(1, array([[0.16312231, 0.83687776]], dtype=float32))


In [ ]:
from scipy.special import softmax
m = softmax([[230   , 1000]])
print(m)

[[0. 1.]]


In [ ]:
kaggle datasets download -d thanhlong1997/word2vec-vn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(data[1000])

{'text': '* Hàng cực hot hoàn thiện đẹp trước tết về ở luôn chỉ 1 căn duy nhất . LH xem nhà : A Thủy 0939965555/0379500000 ... (Tặng 2 phiếu bốc thăm trúng xe Vision trị giá 40tr khi mua nhà của cty chúng tôi) ... * Nhà Phú Lãm xây 3 tầng 35m2 ngõ thông ô tô đỗ 10m đã hoàn thiện giá chỉ 1,63 tỷ (ảnh thật 100%) ... Thiết kế gồm : . Tầng 1 : Phòng khách , bếp , 1 nhà vệ sinh ... Tầng 2 : 2 phòng ngủ , 1 vệ sinh ... Tầng 3 : 1 phòng thờ (phòng ngủ), 1 sân phơi rộng thoáng ... Thiết kế và xây dựng do kiến trúc sư tính toán tối đa công năng sử dụng và rất tiện ích ... Nhà được xây có móng riêng , cột riêng , tường riêng . Đảm bảo an toàn và sửa chữa . Có thể xây thêm tầng 4,5 thoải mái ... Sổ đỏ và giấy phép xây dựng đầy đủ ... * Nội thất cao cấp đầy đủ nội thất cơ bản về chỉ việc ở như hệ thống điện nước , tủ bếp , vệ sinh , trần thạch cao .. ... * Vị trí và tiện ích là 2 yếu tố chính mà mỗi người mua nhà đều quan tâm nhất thì ở đây đáp ứng được đầy đủ các yêu cầu đó ... Nhà gần Quốc lộ 21

In [ ]:

class Model(object):
  def __init__(self):
    tf.reset_default_graph()
    self.n_hidden_1 = 256
    self.n_hidden_2 = 64
    self.dimention_1 = 26 # dimention of vocabulary
    self.dimention_2 = 7 # dimention of entities
    self.connect_dimention = 16 # dimention connect
    self.num_class = 2
    self.keep_prob = 1
    

      # tf Graph input
    self.X_11 = tf.placeholder('float',[None,self.dimention_1],name='X_11')
    self.X_21 = tf.placeholder('float',[None,self.dimention_2],name='X_21')
    self.Y1   = tf.placeholder('float',[None,self.num_class],name='Y1')

    self.weights = {
        'concat_1': tf.Variable(tf.random_normal([self.dimention_1,self.connect_dimention]),name='concat_1'),
        'concat_2': tf.Variable(tf.random_normal([self.dimention_2,self.connect_dimention]),name='concat_2'),
        'h_1'     : tf.Variable(tf.random_normal([self.connect_dimention,self.n_hidden_1]),name='h_1'),
        'h_2'     : tf.Variable(tf.random_normal([self.n_hidden_1,self.n_hidden_2]),name='h_2'),
        'out'     : tf.Variable(tf.random_normal([self.n_hidden_2,self.num_class]),name='out')
    }

    self.biases = {
        'b_1' : tf.Variable(tf.random_normal([self.n_hidden_1]),name='b_1'),
        'b_2' : tf.Variable(tf.random_normal([self.n_hidden_2]),name='b_2'),
        'out' : tf.Variable(tf.random_normal([self.num_class]),name='out')
    }
    self.predict, self.prob = self.neural_net(self.X_11,self.X_21)
    self.initialize_and_restore_session()

  def neural_net(self,x1,x2):
    # Concat Feature
    concat  = tf.add(tf.matmul(x1,self.weights['concat_1']),tf.matmul(x2,self.weights['concat_2']))
    # hidden layer 1
    layer_1 = tf.add(tf.matmul(concat,self.weights['h_1']),self.biases['b_1'])
    layer_1 = tf.nn.relu(layer_1)
    # hidden layer 2
    layer_2 = tf.add(tf.matmul(layer_1,self.weights['h_2']),self.biases['b_2'])
    layer_2 = tf.nn.relu(layer_2)
    # drop out
    drop_out = tf.nn.dropout(layer_2,self.keep_prob)
    # out layer
    out_layer = tf.add(tf.matmul(drop_out,self.weights['out']),self.biases['out'])
    probabilities = tf.nn.softmax(out_layer, axis=-1)
    return out_layer , probabilities

  def initialize_and_restore_session(self):
        """Defines self.sess and initialize the variables"""
        print("Initializing tf session")
        # config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True, device_count={'GPU': 0})
        # config.gpu_options.per_process_gpu_memory_fraction = 0.3
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        self.saver = tf.train.Saver()
        self.saver.restore(self.sess, '/content/model/model145.ckpt')
        print("Complete restore model from ")

  def predict_raw_text(self,input):
    # print(input['text'])
    # text = input['text']
    # graph_meta = tf.train.import_meta_graph('/content/model/model95.ckpt.meta')
    predict = tf.argmax(self.predict, 1)
    vector1 = transform_document_to_vec(input)
    vector2,slot = transform_entity_to_vec(input)
    vector2 = vector2.reshape([1,7])
    pred , prob = self.sess.run([predict,self.prob],feed_dict={self.X_11: vector1,
                                        self.X_21: vector2})
    print(pred , prob)
    return pred


# print(predict_raw_text('Bán gấp nhà Văn Điển 5 tầng ô tô đỗ gần chỉ 2,7 tỷ ... + Nhà mới đẹp Văn Điển ở ngay , ô tô đỗ gần , công năng đầy đủ cho 1 hộ gia đình ở : 1 khách , bếp , 2 ngủ , 3wc , phòng thờ , sân phơi . Có thể lên thêm 2 tầng thoải mái ... + Nhà Văn Điển gần rất nhiều tiện ích : chợ , trung tâm thương mại , bệnh viện , trung tâm thể dục thể thao , huyện ủy thanh trì ... + Khu vực Thanh Trì sắp lên Quận giá trị bất động sản ngày một tang , khách mua năm sbawts tình hình mua đợt này dịch giá có bớt cho ai thiện chí ... + Khu vực Thanh trì lên trung tâm thành phố rất gần chỉ 15 phút , đi các quận rất thuận tiện , về quê qua bến xe nước ngầm chỉ 2 phút ... + Nhà đẹp sổ vuông nở hậu ... + Liên hệ Mrs Thúy : 0965 - 2535 - 83'))
# 1/0



In [ ]:
model = Model()
print(model.predict_raw_text(data[1002]))

Initializing tf session
INFO:tensorflow:Restoring parameters from /content/model/model145.ckpt
Complete restore model from 
[0] [[1. 0.]]
[0]


In [ ]:
print(transform_entity_to_vec(data[1001]))
print(transform_entity_to_vec(data[1002]))

(array([1., 0., 0., 0., 0., 0., 1.]), {'house_sizes': {'use_area': 140.0, 'total_land_area': 35.0, 'contruction_area': 35.0}, 'house_description': {'floor': 4.0, 'mat_tien': '', 'size_mat_ngo': '', 'number_livingroom': '', 'number_kitchen': '', 'number_dressroom': '', 'number_bedroom': '3 phòng ngủ'}, 'land_description': {}, 'price': '1,53 tỷ (Có thương lượng và bao phí sang tên)', 'house_locate': {'near_places': 'chợ Xốm The Vesta chợ Xốm đường Quang Lãm - Phường Phú Lãm - Quận Hà Đông - TP . Hà Nội ngã 3 Ba La ', 'location': 'Quang Lãm '}, 'contact': {'contact_mobile': '0866766916', 'email': ''}})
(array([1., 0., 1., 1., 1., 1., 1.]), {'house_sizes': {'use_area': 144.0, 'total_land_area': 36.0, 'contruction_area': 36.0}, 'house_description': {'floor': 4.0, 'mat_tien': '', 'size_mat_ngo': 'gần 3m', 'number_livingroom': 'Phòng khách', 'number_kitchen': 'bếp', 'number_dressroom': 'vệ sinh', 'number_bedroom': 'Mỗi tầng'}, 'land_description': {}, 'price': '1,52 tỷ thương lượng nhẹ', 'hous

In [ ]:
import json
with open('/content/result4700.json','r',encoding='utf-8') as file:
  data_test = json.load(file)
result = []
for index in range(1000,1100):
  result.append(predict_raw_text(data[index]))

print(result)

In [ ]:
print(result[0])

1


In [ ]:
for index in range(len(result)):
  if result[index] == 1:
    print(index+1000)

1018
1049
1061
1067
1069
1073
1085


In [ ]:
query = pd.read_csv('qq4.csv',encoding='latin-1')

In [ ]:
query.head(5)

,Unnamed: 0,0
0,0,360 giai phong
1,1,4s linh dong
2,2,ascent lakeside
3,3,ban an phu shop villa
4,4,ban biet thu a chau vung tau


In [ ]:
import re
list_index = []
for index in (query.index):
  if re.search('chung cu|chung cư|nhà|nha|biet thu|biệt thự|liền kề|lien ke|đất|dat|căn hộ|can ho|khach san|cua hang|phong tro|mat bang',query['0'][index]):
    list_index.append(index)

In [ ]:
print(len(list_index))

860


In [ ]:
for index in query.index :
  if index not in list_index:
    print(query['0'][index])

360 giai phong 
4s linh dong
ascent lakeside
ban an phu shop villa
ban can shophouse vinhomes smart city dai mo
ban dao tai ho thac ba yen bai
ban homestay da lat
ban ki ot hh1hh2 linh dam hoang mai
ban kiot gia re quang nam
ban penthouse quan 7 gia re
ban san thuong mai du an an binh city
ban sarimi 3pn
ban shophouse geleximco le trong tan
ban shophouse hc golden city
ban shophouse kien hung ha dong
ban trang trai duong lang hoa lac
bat dong san
bat dong san .com
bat dong san .com
bat dong san an giang
bat dong san ben tre
bat dong san bien hoa
bat dong san cham com
bat dong san dien khanh
bat dong san dong anh
bat dong san hai phong
bat dong san hoi an
bat dong san tay ninh
bat dong san thai binh
bat dong san thanh oai ha noi
bat dong san vinh phuc
bat dong san. vn
bat dong san.com.vn
batdongsan
batdongsan da nang
batdongsan. com.vn
batdongsan.com.vn apk
batdongsan.vn
batdongsandanang
bau bang
bds.com
binh duong
C14 kim lien
Can ho c14 kim lien
celadon city
chdv
cho thue phong studio

In [ ]:
import re
class classifi_property(object):
  def __init__(self):
    pass 
  
  def rule_check_type(self,input):
    input = input.lower()
    if re.search('chung cu|chung cư|can ho|căn hộ',input):
      return 'Apartment'
    if re.search('nhà|nha|lien ke|liền kề', input):
      return 'House'
    if re.search('biet thu|biệt thự|villa', input):
      return 'Villa'
    if re.search('khach san|khách sạn', input):
      return 'Hotel'
    if re.search('cửa hàng|cua hang|quan|shop', input):
      return 'Shop'
    if re.search('đất|dat|mặt bằng|mat bang', input):
      return 'Land'
    return 'Unknow'
  
  def rule_check_intent(self,input):
    input = input.lower()
    if re.search('ban|bán|sang nhuong|sang nhượng',input):
      return 'Sell'
    if re.search('mua',input):
      return 'Buy'
    if re.search('thue|thuê',input):
      return 'Rent'
    return 'Unknow'


In [ ]:
model = classifi_property()
for index in query.index:
  if model.rule_check_type(query['0'][index]) == 'Unknow':
    print(query['0'][index])

360 giai phong 
4s linh dong
ascent lakeside
ban dao tai ho thac ba yen bai
ban homestay da lat
ban ki ot hh1hh2 linh dam hoang mai
ban san thuong mai du an an binh city
ban sarimi 3pn
ban trang trai duong lang hoa lac
bat dong san
bat dong san .com
bat dong san .com
bat dong san an giang
bat dong san ben tre
bat dong san bien hoa
bat dong san cham com
bat dong san dien khanh
bat dong san dong anh
bat dong san hai phong
bat dong san hoi an
bat dong san tay ninh
bat dong san thai binh
bat dong san thanh oai ha noi
bat dong san vinh phuc
bat dong san. vn
bat dong san.com.vn
batdongsan
batdongsan da nang
batdongsan. com.vn
batdongsan.com.vn apk
batdongsan.vn
batdongsandanang
bau bang
bds.com
binh duong
C14 kim lien
celadon city
chdv
cho thue phong tro gan etown
cho thue phong tro gan etown cong hoa
cho thue phong tro gan kinh te quoc dan
cho thue phong tro gan truong dai hoc bach khoa lien chieu
cho thue phong tro gia re
cotec phu xuan
de capella
du an diamond riverside
du an dragon city 

In [ ]:
for index in query.index:
  if model.rule_check_intent(query['0'][index]) == 'Unknow':
    print(query['0'][index])

360 giai phong 
4s linh dong
ascent lakeside
bat dong san
bat dong san .com
bat dong san .com
bat dong san an giang
bat dong san ben tre
bat dong san bien hoa
bat dong san cham com
bat dong san dien khanh
bat dong san dong anh
bat dong san hai phong
bat dong san hoi an
bat dong san tay ninh
bat dong san thai binh
bat dong san thanh oai ha noi
bat dong san vinh phuc
bat dong san. vn
bat dong san.com.vn
batdongsan
batdongsan da nang
batdongsan nha trang
batdongsan. com.vn
batdongsan.com.vn apk
batdongsan.vn
batdongsandanang
bds.com
biet thu ecopark
biet thu nam an khanh
biet thu vinhomes riverside
binh duong
C14 kim lien
can ho binh duong
Can ho c14 kim lien
can ho centana thu thiem
can ho cityland go vap
can ho feliz en vista
can ho gold view
can ho hoang anh thanh binh
can ho idico tan phu
can ho moonlight thu duc
can ho opal garden
can ho pegasuite
can ho quan 10
can ho quan 4
can ho quan 8
can ho sky 9
can ho sunrise city
celadon city
chdv
chung cu 1050 chu van an
chung cu 155 nguyen

In [ ]:
a = classifi_property(query['0'][6])
print(a.rule_check)

<bound method classifi_property.rule_check of <__main__.classifi_property object at 0x7fcdf741c240>>
